In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#import matplotlib.axes as axes
import re
import os
import calendar

In [2]:
### FLÜSSE UND MESSSTATIONEN AUSLESEN UND SORTIEREN

# eine leere Liste erstellen, um die Werte zu speichern
df_rv=pd.DataFrame()

# Einlesen der Datei und Sortierung nach den Messstellen-IDs
dateipfad = 'm_2.txt'
data=pd.read_csv(dateipfad, sep=';')
data.sort_values(by='grdc_no').reset_index()
print(data)

# erstellt eine Liste mit den Messstellen-IDs als String
mst_valid_ids = data['grdc_no'].tolist()
mst_valid_lds=[str(i) for i in mst_valid_ids]

# erstellt eine Liste mit den Flussnamen als String
river_ids=data['river'].tolist()
river_ids=[str(i) for i in river_ids]

# erstellt eine Liste mit den Stationsnamen
label_ids=data['station'].tolist()

# Speichern des DataFrames "data" unter neuem Namen (but why?)
df_ids=data


for mst_id in mst_valid_ids:
    try:
        # Generieren eines Dateinamens für jede Stations-ID
        path= f'{mst_id}_monthly.txt'
        data = pd.read_csv(path, sep=';')
        # Die Werte der Spalte 'wasserstd_m' extrahieren und in den Dataframe "df_rv" einfügen
        df_rv[mst_id] = data['wasserstd_m']
    except Exception as e:
        print(f'{mst_id} no found!')

        
print(df_rv.columns.tolist())
print(river_ids)

print(df_rv)

   grdc_no        river                 station  name_in Pegelonline
0  6335050  RHINE RIVER             DUESSELDORF            Dsseldorf
1  6340180   ELBE RIVER  MAGDEBURG-STROMBRUECKE  magdeburgstrombrcke
[6335050, 6340180]
['RHINE RIVER', 'ELBE RIVER']
           6335050     6340180
0      1880.000000  397.000000
1      2120.000000  388.000000
2      2080.000000  376.000000
3      1960.000000  383.000000
4      1740.000000  389.000000
...            ...         ...
45518  1245.479167  157.041667
45519  1189.291667  153.156250
45520  1155.645833  147.541667
45521  1124.395833  141.895833
45522  1106.071429  140.341463

[45523 rows x 2 columns]


In [3]:
### JÄHRLICHER THRESHOLD

threshold=70
p_threshold=(100-threshold)/100

# leerer DataFrame
df_mt_rv=pd.DataFrame()
plot=False

# für jede Spalte im DataFrame df_rv -> jeder Fluss mit dazugehörigen Werten
for j in df_rv.columns:
    # Datenreihe über die gesamte Länge von df_rv (periods), mit Tagen ('D') als Zeitschritte
    months = pd.date_range(start='1901-01-01', periods=len(df_rv), freq='D') 
    # nur die Abflusswerte aus der aktuellen Spalte j aus df_rv als Liste
    data=df_rv[j].tolist()
    # neuer DataFrame, wo wir die Datumsreihe mit den Werten aus df_rv kombinieren
    df = pd.DataFrame({'Date': months, 'Value': data})
    # Datumsspalte als Index setzen
    df['Date']=pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    # das Jahr und der Tag des Jahres wird angegeben (aber doppelt, warum?)
    df['D'] = df.index.dayofyear
    df['year']=df.index.year
    df['day_of_year'] = df.index.dayofyear
    
    #print(df)
    #print(df[df['D']==1])
    #print(df.index.dayofyear)
    
    # Berechnung des 70. Perzentils für jeden Monat

    # alle NaN-Wertwerte werden durch '-999' ersetzt und dann gelöscht
    df['Value'] = df['Value'].fillna(-999)
    df_valid=df[df['Value']!=-999]
    #print(df[df['Value']==-999])
    # könnte man auch dadurch ersetzen:
    #df_valid = df.dropna(subset=['Value'])
    #print(df_valid)

    # Daten in df jahrweise gruppieren, für jedes Jahr den Mittelwert berechnen (['Value'].mean()) und dann davon das Quantil bestimmen (quantile(p_threshold))
    mt = df_valid.groupby('year')['Value'].mean().quantile(p_threshold)
    df_mt_rv[j]=[mt]

print(df_mt_rv)


       6335050     6340180
0  1860.773568  442.532603


In [4]:
### DÜRRE BERECHNEN

import datetime

def calculate_drought(q_datas, MT):
    #Listen, die die Eigenschaften der Dürreereignisse speichern
    ID=[]
    events=[]  # 0,1,2,3
    duration=[]  # unit = month
    mean_deficit=[]  # mean_deficit kleiner als 0
    max_deficit=[]
    sum_deficit=[]
    start_time=[]
    end_time=[]
    deficit_list=[]
    #duration_list=[]
    deta=[]
    Qs=[]
    
    # die Daten aus q_datas in eine Liste überführen
    amean=q_datas.tolist()
    #print(amean)
    # Datenreihe über die gesamte Länge von 'amean' (periods), mit Tagen ('D') als Zeitschritte
    days = pd.date_range(start='1901-01-01', periods=len(amean), freq='D')
    # neuer DataFrame 'df_data' mit den Spalten 'Datum' und 'Value'
    df_data = pd.DataFrame({'Date': days, 'Value': amean})
    # Datumsspalte als Index in 'df_data' setzen
    df_data['Date']=pd.to_datetime(df_data['Date'])
    df_data.set_index('Date', inplace=True)
    
    # drought flag
    dflag=False 
    i = 0
    index_a = 0
    # Fehlwerte (-999) durch einen sehr hohen Wert (999999) ersetzen, damit sie nicht als Dürre erkannt werden
    amean=[999999 if int(i) == -999 else i for i in  amean] 
    s_time = 0  # Startzeitpunkt des Dürreereignisses
    d = 0  # Dauer des Dürreereignisses
    
    # Prüfen von jedem Tag in 'amean'
    for data in amean:
        
        MNQ95 = MT
       
        if data < MNQ95:
            if dflag == False:  # Beginn einer neuen Dürre
                dflag=True 
                ID.append(i)  # der Dürre eine ID geben
                i=i+1
                deta.append(index_a-(s_time+d))
                s_time = index_a  # setzt die Startzeit
                start_time.append(s_time)
                #print(s_time)
                d=1
                Q=[]  # der Abfluss wird gespeichert
                Q.append(data)
                deficit=[]  # das Defizit wird gespeichert
                deficit.append(MNQ95-data)
            elif dflag==True:  # während der Dürre
                # den aktuellen Abfluss und das Defizit speichern
                deficit.append(MNQ95-data)
                Q.append(data)
                d=d+1
                
        elif data >= MNQ95:
            if dflag == True:  # Ende einer Dürre
                duration.append(d)
                mean_deficit.append(sum(deficit)/len(deficit))
                sum_deficit.append(sum(deficit))
                max_deficit.append(max(deficit))
                deficit_list.append(deficit)
                end_time.append(s_time+d)
                Qs.append(min(Q))
                dflag=False
    
        # für den letzten Tag in unserer Datenreihe, weil der sonst verloren gehen würde wenn er Teil einer Dürre ist
        if data == amean[-1]:
            if dflag == True:
                duration.append(d)
                mean_deficit.append(sum(deficit)/len(deficit))
                sum_deficit.append(sum(deficit))
                max_deficit.append(max(deficit))
                deficit_list.append(deficit)
                end_time.append(s_time+d)
                Qs.append(min(Q))
                dflag=False
                
        '''if df_data.index.is_month_end[index_a]:
            if dflag==True:
                duration.append(d)
                mean_deficit.append(sum(deficit)/len(deficit))
                sum_deficit.append(sum(deficit))
                max_deficit.append(max(deficit))
                end_time.append(s_time+d)
                Qs.append(min(Q))
      
                dflag=False'''
    
        index_a = index_a+1

    from datetime import date
    from dateutil.relativedelta import relativedelta  
    
    # Umwandlung des Index in ein richtiges Datum ("Tage seit 1901-01-01")
    def m2y(m):
        startdatum = date(1901, 1, 1)  # Reference start date
        neudate = startdatum + relativedelta(days=m)  # Add months
        years = neudate.year - startdatum.year  # Calculate years
        months = neudate.month - startdatum.month  # Calculate remaining months
    
        if months < 0:  # Handle negative months (when crossing a year)
            years -= 1
            months += 12
        years=1901+years
        months=months+1
        return years, months, neudate  # Return as (years, months)
    
                
    # Berechnung der Start- & Enddaten der Dürreereignisse
    time_months=[m2y(i)[1] for i in start_time]    
    time_years=[m2y(i)[0] for i in start_time]  
    time_date=[m2y(i)[2] for i in start_time]
    end_date=[m2y(i)[2] for i in end_time]
    
    # einen DataFrame erstellen
    df=pd.DataFrame()
    df['ID']=ID
    df['duration']=duration
    df['time_deta']=deta  # [i if i<=365 else 0 deta]
    df['mean_deficit']=mean_deficit
    df['max_deficit']=max_deficit
    df['sum_deficit']=sum_deficit
    df['list_deficit']=deficit_list
    df['Qmin']=Qs
    df['start_date']=time_date
    df['end_date']=end_date
    df['start_year']=time_years
    df['start_month']=time_months
    
    return df

# for i in range(len(ID)):
#     print('ID=',ID[i])
#     print('event type=',events[i])
#     print('duration=',duration[i])
#     print('mean deficit',mean_deficit[i])
#     print('max deficit',max_deficit[i])
#     print('start time=',m2y(start_time[i]))
#     print('\n')

In [5]:
### ERGEBNIS AUSGEBEN LASSEN

# DataFrame erstellen
df_result=pd.DataFrame()

# die beiden Listen df_rv.columns und river_ids so kombinieren, dass für jede Iteration i den aktuellen Spaltennamen und j die entsprechende Fluss-ID erhält
for i,j in zip(df_rv.columns,river_ids):
    # ersetzen der NaN-Werte in df_rv durch -999 als int
    df_rv[i] = df_rv[i].fillna(-999).astype(int)
    # aufrufen der vorher definierten Funktion "calculate_drought" für die aktuelle Spalte i
    # df_mt_rv[i].iloc[0]: ruft aus der Spalte i in 'df_mt_rv' den ersten Wert auf
    df_ssi_c1=calculate_drought(df_rv[i],df_mt_rv[i].iloc[0])

    # erstellen von neuen Spalten im DataFrame 'df_ssi_c1' -> für jede Zeile Name der Station / Fluss
    df_ssi_c1['Station']=[i]*len(df_ssi_c1)
    df_ssi_c1['River']=[j]*len(df_ssi_c1)
    # den DataFrame df_ssi_c1 an das Ende des bereits existierenden DataFrames df_result anhängen
    df_result = pd.concat([df_result, df_ssi_c1], ignore_index = True)

# Dürreereignisse mit einer Dauer von mindestens 5 Tagen
#df_result=df_result[df_result['duration']>=5]

# Dürreereignisse mit einem Gesamtdefizit von mindestens 5
#df_result=df_result[df_result['sum_deficit']>=5]

# eine Kopie von 'df_result' speichern
df_result0 = df_result
# die Datei im zugehörigen Ordner speichern
df_result.to_csv(f'rv_mst_result/FTM_{threshold}/drought_result_all.csv',sep=';')

# Entfernen von doppelten Fluss-IDs
river_id=list(set(river_ids))

#print(df_result)
#print(df_result0)
print(df_result.columns.tolist())


['ID', 'duration', 'time_deta', 'mean_deficit', 'max_deficit', 'sum_deficit', 'list_deficit', 'Qmin', 'start_date', 'end_date', 'start_year', 'start_month', 'Station', 'River']


In [8]:
### ERGEBNISSE NACH SAISONEN

# die beiden Listen df_rv.columns und river_ids so kombinieren, dass für jede Iteration k den aktuellen Spaltennamen und j die entsprechende Fluss-ID erhält
for k,j in zip(df_rv.columns,river_ids):
    # leeren DataFrame erstellen
    df_result_all = pd.DataFrame()
    # Erstellen einer Liste mit allen Jahren, in denen eine Dürre stattgefunden hat
    years=df_result['start_year'].unique().tolist()
    events=[]

    # den DataFrame so filtern, dass nur noch Daten für die aktuelle Station 'k' beibehalten werden
    df_result0=df_result[df_result['Station']==k]
    
    for y in years:
        # neue DataFrames jeweils nur für das aktuelle Jahr 'y'
        df_result_s=pd.DataFrame()
        df_year_1=pd.DataFrame()
        df_year_2=pd.DataFrame()
        #df_year=df_result0[df_result0['start_year']==y]
        # Spalte die das aktuelle jahr enthält im DataFrame 'df_result_s'
        df_result_s['year']=[y]

        # jede 'season' erhält einen Namen und jedes i eine Liste von Monaten
        for season,i in zip(['winter','spring','summer','autumn'],[[11, 12, 1],[2,3, 4],[5, 6, 7],[8, 9, 10]]):
                # den DataFrame so filtern, dass nur noch Ereignisse des aktuellen Jahres 'y' beibehalten werden
                df_year_1=df_result0[df_result0['start_year']==y]
                
                if season == 'winter':
                    # es werden nur Dürreereignisse mit einem Startmonat '1' (Januar) beibehalten
                    df_1=df_year_1[df_year_1['start_month'].isin([1])]
                    if y>1901:
                        # Startmonate 11 & 12 aus dem vorherigen Jahr 'y-1' in den DataFrame integrieren
                        df_year_2=df_result0[df_result0['start_year']==y-1]
                        df_2=df_year_2[df_year_2['start_month'].isin([11,12])]
                        # neuer DataFrame mit dem gesamten Winter
                        df_season=pd.concat([df_1, df_2], ignore_index=True)
                    else:
                        df_season=df_1
                else:
                    df_season=df_year_1[df_year_1['start_month'].isin(i)]

                # alles auf 0 setzen, falls df_season keine Einträge enthält
                if len(df_season) == 0:
                    df_result_s[f'{season}_cumulative_events']=[0]
                    df_result_s[f'{season}_cumulative_duration']=[0]
                    df_result_s[f'{season}_cumulative_deficit']=[0]
                    df_result_s[f'{season}_max_deficit']=[0]
                else:
                    df_result_s[f'{season}_cumulative_events']=[len(df_season)]   # Anzahl der Dürreereignisse
                    df_result_s[f'{season}_cumulative_duration']=[df_season['duration'].sum()]  # Summe der Dauer der Dürreereignisse
                    df_result_s[f'{season}_cumulative_deficit']=[df_season['sum_deficit'].sum()]  # Summe der Defizite
                    df_result_s[f'{season}_max_deficit']=[df_season['max_deficit'].max()]  # maximales Defizit

        # Zusammenfügen aller Ergebnisse
        df_result_all = pd.concat([df_result_all, df_result_s], ignore_index=True)

    df_result_all.to_csv(f'rv_mst_result/FTM_{threshold}/seasonal_drought_result_{j}_{k}.csv',sep=';')